# 13. 연산자 오버로딩: 제대로 하기

연산자 오버로딩은 사용자 정의 객체가 `+`와 `|` 같은 중위 연산자, `-`와 `~`와 같은 단항 연산자를 사용할 수 있게 해줍니다. 

이 장에서는 __단항 연산자__와 __중위 연산자__만 다룹니다.

### 이장에서 다룰 내용

- 파이썬이 다른 자료형의 피연산자로 중위 연산자를 지원하는 방법
- 다양한 자료형의 피연산자를 다루기 위한 덕 타이핑이나 명시적인 자료형 검사의 사용
- `==`, `>`, `<=` 등 향상된 비교 연산자의 별난 행동
- 피연산자를 처리할 수 없다고 중위 연산자 메서드가 알려주는 방법
- `+=`과 같은 계산 할당 연산자의 기본 처리 방식 및 오버로딩 방법

In [1]:
import sys
sys.path.append('../codes')

## 13.1 연산자 오버로딩 기본 지식

__연산자 오버로딩을 혐오하는 사람도 많습니다.__

- 남용될 가능성
- 프로그래머를 혼란스럽게 만들 수 있습니다. 
- 버그를 만들 수 있습니다. 
- 예상치 못한 성능상의 병목이 될 수 있음

__잘 사용한 경우 장점__

- 코드의 가독성이 향상
- 만족스러운 API를 구현
- 다음과 같은 제한을 통해 융통성, 사용성, 안전성을 적절히 유지 가능
    - 내장 자료형에 대한 연산자는 오버로딩할 수 없다. 
    - 새로운 연산자를 생성할 수 없으며, 기존 연산자를 오버로딩만 할 수 있음
    - is, and, or, not 연산자는 오버로딩할 수 없습니다. (그러나 `&`, `|`, `~` 비트 연산자는 가능)


## 13.2 단항 연산자 

- `-`(`__neg__`)
    - 단항 산술 부정. x가 -2면 -x는 2다
    
- `+`(`__pos__`)
    - 단항 산술 덧셈. 일반적으로 x와 +x는 동일하지만, __그렇지 않은 경우도 있습니다.__ 
    
- `~`(`__invert__`)
    - 정수형의 비트 반전. `~x`는 `-(x+1)`로 정의된다 ( `~x == -(x+1))`

파이썬 참조 문서에서는 내장 함수인 `abs()`도 단항연산자로 나열합니다. 

단항 연산자는 `self` 인수 하나를 받는 적절한 특별 메서드를 구현하면 됩니다. 

클래스에 논리적으로 합당한 연산을 수행해야 하고 __새로운 객체를 반환__해야하는 연산자의 핵심 규칙을 지켜야 합니다. __즉 `self`를 수정하지 말고 객체를 새로 생성해야 합니다.__

In [2]:
# abs()의 경우 스칼라형 숫자가 결과로 나와야 합니다.
def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

# 새로운 Vector 객체를 만들고 self의 모든 요소를 반댓값으로 채운다.
def __neg__(self):
    return Vector(-x for x in self)

# 새로운 Vector 객체를 만들고 self의 모든 요소로 채웁니다. 
def __pos__(self):
    return Vecotr(self)

In [3]:
import math
import reprlib
import numbers
import operator
import functools
from array import array

class Vector:
    
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes)
    
    # abs()의 경우 스칼라형 숫자가 결과로 나와야 합니다.
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    # 새로운 Vector 객체를 만들고 self의 모든 요소를 반댓값으로 채운다.
    def __neg__(self):
        return Vector(-x for x in self)

    # 새로운 Vector 객체를 만들고 self의 모든 요소로 채웁니다. 
    def __pos__(self):
        return Vector(self)
        
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attributes {attr_name!r}'
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name = name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(slef):
        return (self.angle(n) for n in range(1, len(slef)))
    
    def __format__(slef, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(coponents))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:], cast(typecode))
        return cls(memv)

In [4]:
v1 = Vector([1,2])

In [5]:
+v1

Vector([1.0, 2.0])

In [6]:
-v1

Vector([-1.0, -2.0])

In [7]:
abs(v1)

2.23606797749979

In [8]:
~v1

TypeError: bad operand type for unary ~: 'Vector'

### `-x`와 `+x`가 동일하지 않은 경우

누구나 x와 +x가 같을 것이라고 생각합니다.(`x==+x`). 사실 파이썬에서는 거의 항상 똑같습니다. 그러나 표준 라이브러리 안에서 x와 +x가 다른( `x != +x`) 두 가지 사례를 발견했습니다. 

#### 1. `decimal.Decimal`
- Decimal 객체 x를 생성하고 다르게 생성된 콘텍스트에서 +x를 평가하면 x와 +x가 달라질 수 있습니다.
- 특정 정밀도로 객체 x를 생성 후 정밀도를 바꾸면 달라질 수 있습니다.

In [9]:
## ex.13.2
import decimal

# 현재 산술 콘텍스트 전역 설정에 대한 참조를 가져옵니다.
ctx = decimal.getcontext()

# 산술 콘텍스트의 정밀도를 40으로 설정
ctx.prec = 40

# 현재 정밀도를 이용해서 1/3을 계산
one_third = decimal.Decimal('1') / decimal.Decimal('3')

print('\nBefore : ')
print(one_third == +one_third)
print(one_third)
ctx.prec = 28
print('\nAfter : ')
print(one_third == +one_third)
print(one_third)


Before : 
True
0.3333333333333333333333333333333333333333

After : 
False
0.3333333333333333333333333333333333333333


#### 2. `collections.Counter`

- Counter 클래스는 두 Counter 객체의 합계를 구하는 중위 연산자 `+`등 여러 산술 연산자를 구현합니다. 

- 실제로 카운터는 음수가 될 수 없으므로, Counter의 덧셈은 음수나 0인 카운터를 버립니다. 또한 `+`는 빈 Counter 객체를 더하는 연산이므로 0보다 큰 값만 유지하는 Counter 객체를 새로 생성합니다. 

In [10]:
## ex.13.3

from collections import Counter

ct = Counter('abcabcabc')
print(ct)
ct['b'] = -3
ct['c'] = 0
print(ct)
print(+ct)

Counter({'a': 3, 'b': 3, 'c': 3})
Counter({'a': 3, 'c': 0, 'b': -3})
Counter({'a': 3})


## 13.3 벡터를 더하기 위해 `+` 오버로딩하기

#### `__add__()` 구현

In [11]:
import itertools

class Vector2(Vector):
    def __add__(self, other):
        # pairs는 self에서 a를, other에서 b를 가져와서 
        # (a,b) 튜플을 생성하는 제너레이터
        # 길이가 다른 경우 fillvalue로 채워줍니다.
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a,b in pairs)

In [12]:
v1 = Vector2([3, 4, 5])
v2 = Vector2([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [13]:
v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [14]:
v1 = Vector2([3, 4, 5, 6])
v3 = Vector2([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

`__add__()` 메서드는 새로운 Vector 객체를 만듭니다. (__`self`나 `other`값을 변경하지 않습니다.__)

In [15]:
## ex.13.5
v1 = Vector2([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [16]:
from vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

#### `__radd__()` 구현

왼쪽 피연산자가 Vector 객체가 아니면 실패하는 `Vector.__add__()`

In [17]:
## ex.13.6
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1

TypeError: can only concatenate tuple (not "Vector") to tuple

서로 다른 객체형에 대한 연산을 지원하기 위해 파이썬은 중위 연산자의 특별 메서드에 특별 디스패치 매커니즘을 구현해야 합니다. 

1. a에 `__add__()` 메서드가 정의되어 있으면 `a.__add__(b)`를 호출, 결과가 `NotImplemented`가 아니면 반환

2. a에 `__add__()` 메서드가 정의되어 있지 않거나, 정의되어 있더라도 호출 후 `NotImplemented`가 반환되면, b에 `__radd__()` 메서드가 정의되어 있는지 확인해서 `b.__add__(a)`를 호출하고, 결과가 `NotImplemented`가 아니면 반환

3. b에 `__radd__()`가 정의되어 있지 않거나, 정의되어 있더라도 호출 후 `NotImplemented`가 반환되면 __지원하지 않는 피연산자형__이라는 메시지와 함께 `TypeError` 발생

> `__radd__()`는 `__add__()`의 역순 버전입니다. 

위의 Error를 처리하기 위해 `__radd__()` 메서드를 만들어 줍니다. 왼쪽 연산자가 `__add__()`를 구현하고 있지 않거나, 오른쪽 피연산자를 처리할 수 없을 때, 최후의 수단으로 오른쪽 피연산자의 `__radd__()`를 활용하게 됩니다.

In [18]:
import itertools

class Vector2(Vector):
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a,b in pairs)
    def __radd__(self, other):
        return self + other

In [19]:
## ex.13.6
v1 = Vector2([3, 4, 5])
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

이런 방식은 교환법칙이 성립하는 모든 연산자에 적용가능합니다.

#### 다양한 에러 처리
앞써 구현한 `__add__()` 메서드들은 비반복형 객체에 적용하는 경우 에러가 발생합니다.

In [20]:
v1 + 1

TypeError: zip_longest argument #2 must support iteration

반복형이더라도 실수형 항목과 덧셈 불가능할 때는 또 에러가 발생합니다.

In [21]:
v1 + 'abc'

TypeError: unsupported operand type(s) for +: 'float' and 'str'

여기서 문제점은 쓸데없는 에러 메시지 발생과, 적절한 결과를 반환할 수 없는 경우 `NotImplemented`값을 반환하지 않고 `TypeError`예외를 발생시킨 겂니다.

덕타이핑 정신을 발휘해 우리는 `other` 객체의 자료형을 검사하기 보단 일단 연산을 시도하고, `NotImplemented`를 반환합시다. 그 후에 `TypeError`예외를 발생시킵니다.

In [25]:
import itertools

class Vector2(Vector):
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a,b in pairs)
        except TypeError:
            return NotImplemented
        
    def __radd__(self, other):
        return self + other

In [27]:
v1 = Vector2([3, 4, 5])
v1 + 'abc'

TypeError: unsupported operand type(s) for +: 'Vector2' and 'str'

## 13.4 벡터를 스칼라와 곱하기 위해 `*` 오버로딩하기

- `Numpy`에서 스칼라곱 지원해줍니다.
- 벡터의 내적을 구하기 위해서는 `numpy.dot()`을 사용합시다

In [29]:
class Vector2(Vector):
    def __mul__(self, scalar):
        return Vector(n * scalar for n in self)
    
    def __rmul__(self, scalar):
        return self * scalar

In [31]:
v1 = Vector2([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [32]:
11 * v1

Vector([11.0, 22.0, 33.0])

이 메서드들은 호환되는 피연산자를 사용하는 한 제대로 작동합니다. 따라서 complex 숫자는 사용할 수 없지만, int, bool, `fractions.Fraction` 객체도 사용할 수 있습니다. 

앞서 덧셈 오버라이딩 처럼 덕 타이핑 기법을 사용해서 `__mul__()` 안에서 `TypeError`를 잡을 수도 있지만, 여기서는 구스 타이핑 기법을 사용해서 명시적인 방법을 사용하는 것이 좋습니다. 

In [33]:
import numbers

class Vector2(Vector):
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    def __rmul__(self, scalar):
        return self * scalar

In [34]:
v1 = Vector2([1., 2., 3.])
14 * v1

Vector([14.0, 28.0, 42.0])

In [35]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [36]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

#### 중위 연산자 메서드명
_인플레이스 연산자는 13.6절에서 자세히 설명합니다._

![중위 연산자 메서드명](../images/13_1.png)

![중위 연산자 메서드명](../images/13_2.png)

#### 파이썬 3.5에 추가된 `@` 연산자

파이썬 3.5버전 이후부터 행렬 곱셈(내적)을 위한 연산자 `@`를 제공합니다. 

In [38]:
class Vector2(Vector):
    
    def __matmul__(self, other):
        try:
            return sum(a*b for a,b in zip(self, other))
        except TypeError:
            return NotImplemented
        
    def __rmatmul__(self, other):
        return self @ other

In [39]:
v1 = Vector2([1, 2, 3])
v2 = Vector2([5, 6, 7])
v1 @ v2

38.0

In [40]:
v1 @ (5, 6, 7)

38.0

In [40]:
v1 @ (5, 6, 7)

38.0